In [ ]:
'''谋篇布局，先确定实现那些模块，每个模块都有哪些功能'''


''' 构建计算图：LSTM模型：
        embedding层
        LSTM层
        FC层
        train_op
        
    训练流程代码：
    数据集封装代码：
        API: next_batch(batch_size)
    词表封装：
        API：sentence2id（text_sentence）:句子转换ID
    类别封装：
        API：category2id（text_category）:类别转ID
'''

In [1]:
import tensorflow as tf
import os
import sys
import numpy
import math

tf.logging.set_verbosity(tf.logging.INFO)#tf中的print日志模块

In [2]:
'''7-10 超参数定义'''
def get_default_params():
    '''这个API可以帮助啊管理模型的所有参数，返回的是一个对象，里面的参数都可以通过对象。参数名字来使用'''
    return tf.contrib.training.HParams(
        num_embending_size = 16,#embending向量长度
        num_timesteps = 50,#指定LSTM步长，一个centent里面有多少词语
        num_lstm_nodes = [32,32],#lstm的size是多少
        num_lstm_lays = 2,#层数  ，有两层每一层都有32个神经单元
        num_fc_nodes = 32,#fc层神经单元数目
        batch_size = 100,
        clip_lstm_grads = 1.0,#控制梯度大小因为lstm很容易发生梯度爆炸（设置上限）和梯度消失（lr_rate解决）等问题
        learning_rate = 0.001,   
        num_word_threshold =  10#统计的额词频filter上限
    )

'''得到默认的参数配置'''
hps = get_default_params()

'''定义输入和输出文件'''
train_file = 'cnews_data/cnews.train.seg.txt'
val_file = 'cnews_data/cnews.val.seg.txt'
test_file = 'cnews_data/cnews.test.seg.txt'
vocab_file = 'cnews_data/cnews.covab.txt'
category_file = 'cnews_data/cnews.category.txt'
output_floder = 'cnews_data/run_text_rnn'

if not os.path.exists(output_floder):
    os.mkdir(output_floder)

In [9]:
'''7-11 词表封装于类别封装'''
class Vocab:
    def __init__(self, filename, num_word_threshold):
        '''读这个文件，将独处的文件的id放到map里面'''
        '''私有变量，不能通过对象直接访问而是要通过函数去访问'''
        self._word_to_id = {}
        self._unk = -1 #单独给出
        self._num_word_threshold = num_word_threshold
        self._read_dict(filename)
        
    def _read_dict(self, filename):
        '''把词转化成id'''
        with open(filename,'r') as f:
            for line in f:
                word, frequence = line.strip('\s\r').split('\t')
                frequence = int(frequence)
                if frequence < self._num_word_threshold:
                     continue
                else:
                    idx = len(self. _word_to_id)
                if word == '<UNK>':
                    self._unk = idx
                self._word_to_id[word] = idx
    
    def word_to_id(self, word):
        '''句子切分的词在字典中不存在'''
        return self._word_to_id.get(word, self._unk)
    
    #给类加一些成员函数:有时候会访问一些unk的id等
    @property
    def unk(self):
        return self._unk
    
    def size(self):
        return len(self._word_to_id)
    
    def centence_to_id(self, centence):
        '''把centence转化为id'''
        word_ids = [self.word_to_id(cur_word) for cur_word in centence.split()]#对句子用空格切分，然后用每一个词在字典中进行查询。但可能有些词并不存在
        return word_ids
    
    
    
    
class  CategoryDict:
    def __init__(self,filename):
        self._categroy_to_id = {}
        with open(filename, 'r') as f:
            for line in f:
                categroy = line.strip('\r\n')
                idx = len(self._categroy_to_id)
                self._categroy_to_id[categroy] = idx
    
    def category_to_id(self,categroy):
        if not categroy in self._categroy_to_id:
            raise Exception('%s is not in our category list' % categroy)
        return self._categroy_to_id[categroy]
        
vocab = Vocab(vocab_file, hps.num_word_threshold)
tf.logging.info('vocab_size:%d'%vocab.size())

category_vocab = CategoryDict(category_file)
test_str = '时尚'
print('label: %s, id: %d' % (test_str,category_vocab.category_to_id(test_str)))

INFO:tensorflow:vocab_size:77311
label: 时尚, id: 5
